In [2]:
%matplotlib inline

import glob
from platform import python_version

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn
# import torch

In [3]:
from sklearn.preprocessing import StandardScaler
from numpy.lib.stride_tricks import sliding_window_view

np.seterr(divide='ignore', invalid='ignore')

def data_generator(data):
    h, w = data.shape
    # print(h, w)

    shape = (6,w)
    new_data = sliding_window_view(data, shape)
    n, *_ = new_data.shape
    new_data = new_data.reshape((n,6,w))
    return new_data

def transform_data(arr):
    x_arr = arr[:,:-1]
    y_arr = arr[:,-1,-3]
    return x_arr, y_arr

#反标准化
def inverse_label(data, width):
    l, *_ = data.shape
    data_trans = np.zeros((l, width))
    data_trans[:,-3] = data.reshape((l))
    data_trans = scaler.inverse_transform(data_trans)
    return data_trans[:,-3]

In [4]:
df = pd.read_csv('per_person_data/out_AS14.23.csv')
df = df.dropna(thresh=2, subset=['activity', 'call', 'mood', 'screen'])
df = df.dropna(how='all', subset=['appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'])
df = df.sort_values('time')
print(df)
df = df.fillna({'call':0,'sms':0, 'appCat.builtin':0, 'appCat.communication':0, 'appCat.unknown':0,
       'appCat.entertainment':0, 'appCat.other':0, 'appCat.social':0, 'appCat.finance':0, 'appCat.game':0, 'appCat.office':0, 'appCat.travel':0, 'appCat.utilities':0, 'appCat.weather':0,  'appCat.weather':0})
# df = df.drop(columns=['appCat.finance', 'appCat.game', 'appCat.office', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'])
df = df.drop(columns=['appCat.unknown'])
df = df.fillna(df.median(numeric_only=True))
# print(df.shape)
# df.drop_duplicates(subset=['time'])
df = df.iloc[: , 1:]
df_no_time = df.select_dtypes(include=['int64','float64'])
print(df_no_time.shape)
# print(df_no_time)
# df_no_time.columns
data = df_no_time.to_numpy()
# data = df_no_time.to_numpy()
# # print(data)
# h, w = data.shape
# # print(h, w)

# shape = (6,w)
# new_data = sliding_window_view(data, shape)

    Unnamed: 0        time  activity  appCat.builtin  appCat.communication  \
27          12  2014-03-22  0.371361       25.241688             24.304156   
28       16968  2014-03-23  0.217996       19.090643             34.981971   
29       16983  2014-03-24  0.355413       26.189233             18.445960   
30       17007  2014-03-25  0.274954       21.629851             30.290984   
31       17031  2014-03-26  0.405049        8.758588             35.726369   
32       17055  2014-03-27  0.356218       18.962623             54.889647   
33       17077  2014-03-28  0.405412       10.331565             29.506020   
34       17101  2014-03-29  0.397176       22.492611             23.053708   
35       17125  2014-03-30  0.418194       14.227364             15.994000   
36       17139  2014-03-31  0.383015       25.317161             38.086881   
37       17163  2014-04-01  0.511297       11.200867             32.766246   
38       17187  2014-04-02  0.469348       27.498537            

In [6]:
# Data Scaling 
# Fill in nan with all dataset's median
from sklearn.impute import SimpleImputer
empty = []
all_data = np.array(empty)

files = sorted(glob.glob('per_person_data/*.csv'))
# files = sorted(glob.glob('test/*.csv'))
for f in files:
    df = pd.read_csv(f)
    df = df.dropna(thresh=2, subset=['activity', 'call', 'mood', 'screen'])
    df = df.dropna(how='all', subset=['appCat.builtin', 'appCat.communication', 'appCat.entertainment', 'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'])
    df = df.sort_values('time')
    # df = df.fillna({'call':0,'sms':0})
    # df = df.dropna(thresh=int(0.5*len(df)), axis=1)
    df = df.fillna({'call':0,'sms':0, 'appCat.builtin':0, 'appCat.communication':0, 'appCat.entertainment':0, 'appCat.other':0, 'appCat.social':0})
    df = df.drop(columns=['appCat.finance', 'appCat.game', 'appCat.office', 'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'])
    # df = df.fillna(df.mean(numeric_only=True))
    df = df.iloc[: , 1:]
    df_no_time = df.select_dtypes(include=['int64','float64'])
    data = df_no_time.to_numpy()
    
    new_data = data_generator(data)
    
    if all_data.size == 0:
        all_data = new_data
    else:
        all_data = np.concatenate((all_data, new_data), axis=0)    

# scale data
scaler = StandardScaler()
nsamples, nx, ny = all_data.shape
all_data = all_data.reshape((nsamples*nx, ny))
im = SimpleImputer(missing_values=np.nan, strategy="mean")
all_data = im.fit_transform(all_data)
trans_data = scaler.fit_transform(all_data).reshape((nsamples, nx, ny))
all_data = all_data.reshape((nsamples, nx, ny))

In [12]:
'''
Data Preprocessing & Feature Engineering Revamped
USE ONLY FOR CONVENTIONAL ML
'''

df_all = pd.DataFrame() 

files = sorted(glob.glob('per_person_data/*.csv'))
for f in files:
    # Initial filtering & preprocessing
    df = pd.read_csv(f)
    df = df[df['mood'].notna()]
    df = df.sort_values('time')
    df.drop_duplicates(subset=['time'])
    df = df.iloc[: , 1:] 


    # NOTE: Turn this on for extra features 
    if True:
        days_back = 5
        mood = np.array(df['mood'])
        df['past_mood'] = [np.nan] * days_back + [np.mean(mood[i:i+days_back]) for i in range(len(mood) - days_back)]
        df['past_mood_std'] = [np.nan] * days_back + [np.std(mood[i: i+days_back]) for i in range(len(mood) - days_back)]

        valence = np.array(df['circumplex.valence'])
        df['past_valence'] = [np.nan] * days_back + [np.mean(valence[i:i+days_back]) for i in range(len(valence) - days_back)]
        df['past_valence_std'] = [np.nan] * days_back + [np.std(valence[i: i+days_back]) for i in range(len(valence) - days_back)]

        arousal = np.array(df['circumplex.arousal'])
        df['past_arousal'] = [np.nan] * days_back + [np.mean(arousal[i:i+days_back]) for i in range(len(arousal) - days_back)]
        df['past_arousal_std'] = [np.nan] * days_back + [np.std(arousal[i: i+days_back]) for i in range(len(arousal) - days_back)]


    df['next_mood'] = df['mood'].shift(periods=-1) # Create target variable
    df = df[:-1]
    df_all = pd.concat([df_all, df], axis=0) # NOTE: SLOW for large dataframes   

df_all = df_all.drop(['time'], axis=1)
# df_all = df_all.drop(['mood'], axis=1)


for col in df_all.columns:
    prc_nan = df_all[col].isnull().sum() / len(df_all[col])
    
    if prc_nan < .4:
        df_all[col] = df_all[col].fillna(df_all[col].median())
    else:
        df_all = df_all.drop([col], axis=1)

# data = df_all.to_numpy()
# all_data = data_generator(data)

# scale data
scaler = StandardScaler()
nsamples, nx, ny = all_data.shape
trans_data = all_data.reshape((nsamples*nx, ny))
trans_data = scaler.fit_transform(trans_data).reshape((nsamples, nx, ny))

df_all

,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.other,appCat.social,circumplex.arousal,circumplex.valence,mood,screen,past_mood,past_mood_std,past_valence,past_valence_std,past_arousal,past_arousal_std,next_mood
7,0.096359,12.361731,38.598403,48.245667,12.069500,54.034429,-0.250000,0.750000,6.250000,65.742128,7.090000,0.417013,0.720000,0.172047,-0.100000,0.316220,6.333333
8,0.096359,12.361731,38.598403,48.245667,12.069500,54.034429,0.000000,0.333333,6.333333,65.742128,7.090000,0.417013,0.720000,0.172047,-0.100000,0.316220,6.200000
26,0.134050,14.269173,55.095526,53.024000,29.968875,75.141667,0.200000,0.200000,6.200000,153.665872,7.090000,0.417013,0.720000,0.172047,-0.100000,0.316220,6.400000
27,0.236880,7.781160,51.697063,46.662000,14.020429,36.636000,0.600000,0.500000,6.400000,47.613651,7.090000,0.417013,0.720000,0.172047,-0.100000,0.316220,6.800000
28,0.142741,12.134396,53.442031,31.448667,12.137167,50.046611,0.200000,0.800000,6.800000,67.067644,7.090000,0.417013,0.720000,0.172047,-0.100000,0.316220,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.115201,7.711316,34.320472,68.876900,16.845500,111.046204,-0.400000,0.000000,5.400000,300.399810,6.046667,0.245719,0.126667,0.197934,-0.500000,0.502770,6.200000
99,0.012704,8.545776,59.382943,62.347000,17.835750,50.259100,-0.600000,0.400000,6.200000,113.607750,5.926667,0.359413,0.166667,0.139443,-0.446667,0.496365,8.200000
100,0.103301,9.590398,37.238750,43.907571,27.891895,99.281685,0.000000,1.200000,8.200000,113.657683,6.016667,0.360247,0.196667,0.167465,-0.666667,0.153116,7.000000
101,0.169354,6.452161,54.388125,68.369000,9.734000,198.880222,-1.333333,1.000000,7.000000,118.978033,6.490000,0.923255,0.436667,0.405024,-0.500000,0.281069,6.800000


In [ ]:
#RNN
import tensorflow as tf
from tensorflow.keras import optimizers, regularizers, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from keras_self_attention import SeqSelfAttention
from attention import Attention

# divide dataset
X, Y = transform_data(trans_data)

#Generate indices on the row-wise length of the whole 
#array
fold_indices = np.arange(X.shape[0])

#Shuffle the indices -- if you want, but not 
#neccessary
np.random.shuffle(fold_indices)

k = 8
#Split the indices into k-parts (returns a list of 
#numpy arrays)
cross_validatoin_score = []
test_all_x = np.array(empty)
test_all_y = np.array(empty)
eval_indices = np.array_split(fold_indices, k)
for idx, e in enumerate(eval_indices):
    #Define the evaluation set for the current fold
    test_set_x = X[e]
    test_set_y = Y[e]

    #exclude the upon parts indices from the 
    #whole array (similarly on the upon answers)
    
    mask_eval = np.ones(X.shape[0], bool)
    
    #Set indices of the eval set to false
    mask_eval[e] = False

    #Subset by the bool array:
    train_set_x = X[mask_eval].astype('float32')
    train_set_y = Y[mask_eval].astype('float32')
    print(f'Fold: {idx}')
    print(f'Train set shape: {train_set_x.shape}    Test set shape: {test_set_x.shape}')
    num_samples, time_steps, input_dim = train_set_x.shape

    model = Sequential()
    model.add(Input(shape=(time_steps, input_dim)))
    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Dropout(rate=0.3))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Bidirectional(LSTM(128,return_sequences=False)))
    model.add(Dropout(rate=0.7))
    # model.add(Attention(units=32))
    # model.add(Dense(4,activation="tanh",  kernel_regularizer=regularizers.l1(0.01)))
    # model.add(Dropout(rate=0.2))
    model.add(Dense(1,activation="tanh",  kernel_regularizer=regularizers.l1(0.01)))

    optimizer = optimizers.Adam(learning_rate=0.01, clipvalue=0.5)
    model.compile(optimizer=optimizer,loss="mean_squared_error", metrics=["mse"])
    model.fit(train_set_x,train_set_y,batch_size=32,epochs=7,validation_split=0.1,verbose=1)
    # Evaluate the model on the test data using `evaluate`
    print("Evaluate on test data")
    results = model.evaluate(test_set_x, test_set_y, batch_size=64)
    # print("test loss, test acc:", results)
    cross_validatoin_score.append(results)

    if test_all_x.size == 0:
        test_all_x = test_set_x
        test_all_y = test_set_y
    else:
        test_all_x = np.concatenate((test_all_x, test_set_x), axis=0)
        test_all_y = np.concatenate((test_all_y, test_set_y), axis=0)

    


_, _, w = train_set_x.shape
#预测测试集
# predict_trainY=model.predict(train_set_x)
predict_testY=model.predict(test_set_x)
#反标准化
# trainY=scaler.inverse_transform(predict_trainY)
testY_real = inverse_label(test_set_y[:,np.newaxis], w)
testY_predict = inverse_label(predict_testY, w)
# testY_real = Y_test
# testY_predict = predict_testY
print(model.summary())
print(np.mean(np.array(cross_validatoin_score),axis=0))

#看一看数据形状
print("Y:",testY_predict.reshape((-1)),testY_predict.shape)
print("Y_real:",testY_real,testY_real.shape)
plt.figure(figsize=(12,8))
plt.plot(testY_predict,"r",label="Predict")
plt.plot(testY_real,"b",label="Real")
plt.legend()
plt.show()


predict_trainY=model.predict(train_set_x)
#反标准化
trainY_real = inverse_label(train_set_y[:,np.newaxis], w)
trainY_predict = inverse_label(predict_trainY,w)

#看一看数据形状
# print("Y:",trainY_predict.reshape((-1)),trainY_predict.shape)
# print("Y_real:",trainY_real,trainY_real.shape)
plt.figure(figsize=(12,8))
plt.plot(trainY_real,"b",label="Real")
plt.plot(trainY_predict,"r",label="Predict")
plt.legend()
plt.show()

In [13]:
# ML LR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
# divide dataset
X, Y = df_all.iloc[:,:-1], df_all.iloc[:,-1]
# nsamples, nx, ny = X.shape
# X = X.reshape((nsamples,nx*ny))
k = 8
lr=LinearRegression()
kf=KFold(n_splits=k, shuffle=True, random_state=0)
score=cross_val_score(lr,X,Y,cv=kf,scoring='neg_mean_squared_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))

cross Validation scores: [-0.36513931 -0.32247513 -0.42611989 -0.34446178 -0.43493267 -0.47517793
 -0.30465195 -0.39567133]
Average Cross Validation score :0.3835787493398479


In [14]:
# ML Rigid
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import RidgeCV
# divide dataset
X, Y = transform_data(trans_data)
nsamples, nx, ny = X.shape
X = X.reshape((nsamples,nx*ny))

ridge=RidgeCV(alphas=np.arange(1,1001,100))
kf=KFold(n_splits=k)
score=cross_val_score(ridge,X,Y,cv=kf,scoring='neg_mean_squared_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))


cross Validation scores: [-0.78866047 -1.20095175 -0.94805789 -0.50118238 -0.38778489 -0.62491104
 -0.71198182 -0.76102679]
Average Cross Validation score :0.7405696288974254


In [145]:
# ML Lasso
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LassoCV
# divide dataset
X, Y = transform_data(trans_data)
nsamples, nx, ny = X.shape
X = X.reshape((nsamples,nx*ny))

lasso=LassoCV()
kf=KFold(n_splits=k, shuffle=False, random_state=0)
score=cross_val_score(lasso,X,Y,cv=kf,scoring='neg_mean_squared_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))

ValueError: Setting a random_state has no effect since shuffle is False. You should leave random_state to its default (None), or set shuffle=True.

In [151]:
# ML SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR

# divide dataset
trans_data = df_all.to_numpy()

k = 8
# divide dataset
X, Y = trans_data[:,:-1], trans_data[:,-1]
print(Y.shape)
# nsamples, nx, ny = X.shape
# X = X.reshape((nsamples,nx*ny))
clf = SVR(kernel='linear')
kf=KFold(n_splits=k, shuffle=True, random_state=0)
score=cross_val_score(clf,X,Y,cv=kf,scoring='neg_mean_squared_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))

(1241,)
cross Validation scores: [-0.31964298 -0.39979828 -0.38949372 -0.4266173  -0.48596002 -0.30108396
 -0.33831627 -0.40304046]
Average Cross Validation score :0.38299412508312575


In [119]:
# ML KNN
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor

# divide dataset
X, Y = transform_data(trans_data)
nsamples, nx, ny = X.shape
X = X.reshape((nsamples,nx*ny))
neigh = KNeighborsRegressor(n_neighbors=2)
kf=KFold(n_splits=k, shuffle=True, random_state=0)
score=cross_val_score(neigh,X,Y,cv=kf,scoring='neg_mean_absolute_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))

cross Validation scores: [-0.61314166 -0.63089991 -0.58751037 -0.62167102 -0.5721789  -0.55402909
 -0.57904142 -0.57310075]
Average Cross Validation score :0.5914466398137868


In [16]:
# ML RandomForestReg
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
X, Y = df_all.iloc[:,:-1], df_all.iloc[:,-1]
# nsamples, nx, ny = X.shape
# X = X.reshape((nsamples,nx*ny))
regr = RandomForestRegressor(max_depth=3, random_state=0, n_estimators=50, criterion='squared_error')
kf=KFold(n_splits=k, shuffle=True, random_state=0)
score=cross_val_score(regr,X,Y,cv=kf,scoring='neg_mean_squared_error')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))


cross Validation scores: [-0.42035397 -0.34562986 -0.41398744 -0.33686016 -0.40716167 -0.48270924
 -0.32012631 -0.4354917 ]
Average Cross Validation score :0.3952900438441695


In [128]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(max_depth = 4,
                             learning_rate = 1e-1,
                             n_estimators = 50,
                             objective = 'reg:squarederror',
                             n_jobs = -1)

kf=KFold(n_splits=k, shuffle=True, random_state=0)
score=cross_val_score(xgb_model,X,Y,cv=kf,scoring='r2')
print("cross Validation scores: {}".format(score))
print("Average Cross Validation score :{}".format(np.negative(score.mean())))


cross Validation scores: [0.30177766 0.32141243 0.18194864 0.22270254 0.05735733 0.18233292
 0.32411622 0.09671577]
Average Cross Validation score :-0.21104543795391273


In [ ]:
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

In [ ]:


X_train = X_train.astype('float32')
Y_train = Y_train.astype('float32')
num_samples, time_steps, input_dim = X_train.shape

model = Sequential()
model.add(Input(shape=(time_steps, input_dim)))
model.add(Bidirectional(LSTM(32,return_sequences=True)))
# model.add(LSTM(16,return_sequences=False))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
# model.add(Attention(name='attention_weight'))
model.add(Dense(4,activation="tanh",  kernel_regularizer=regularizers.l1(0.01)))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation="tanh",  kernel_regularizer=regularizers.l1(0.01)))

optimizer = optimizers.Adam(learning_rate=0.01, clipvalue=0.5)
model.compile(optimizer=optimizer,loss="mean_squared_error", metrics=["mse"])
model.fit(X_train,Y_train,batch_size=64,epochs=7,validation_split=0.1,verbose=1)
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results)
# print(f'Evaluate: {model.evaluate(X_test,Y_test, verbose=1)}')
model.summary()

In [ ]:
#反标准化
def inverse_label(data, width):
    l, _ = data.shape
    data_trans = np.zeros((l, width))
    data_trans[:,-3] = data.reshape((l))
    data_trans = scaler.inverse_transform(data_trans)
    return data_trans[:,-3]

h, _, w = X_train.shape
#预测测试集
predict_trainY=model.predict(X_train)
predict_testY=model.predict(X_test)
#反标准化
# trainY=scaler.inverse_transform(predict_trainY)
testY_real = inverse_label(Y_test[:,np.newaxis], w)
testY_predict = inverse_label(predict_testY, w)
# testY_real = Y_test
# testY_predict = predict_testY


#看一看数据形状
print("Y:",testY_predict.reshape((-1)),testY_predict.shape)
print("Y_real:",testY_real,testY_real.shape)
plt.figure(figsize=(12,8))
plt.plot(testY_predict,"b",label="Predict")
plt.plot(testY_real,"r",label="Real")
plt.legend()
plt.show()

In [ ]:
#看一看数据形状
# trainY_predict = inverse_label(predict_trainY, w)
# trainY_real = inverse_label(Y_train[:,np.newaxis], w)
trainY_predict = predict_trainY
trainY_real = Y_train

print("Y:",trainY_predict.reshape((-1)),trainY_predict.shape)
print("Y_real:",trainY_real,trainY_real.shape)
plt.figure(figsize=(12,8))
plt.plot(trainY_predict,"b",label="Predict")
plt.plot(trainY_real,"r",label="Real")
plt.legend()
plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers,Input,optimizers
# from tensorflow.keras.layers import Bidirectional
# input_shape=Input(shape=(x_train.shape[1],x_train.shape[2]))
# lstm1=layers.LSTM(32,return_sequences=1)(input_shape)
# print("lstm1:",lstm1.shape)
# lstm2=layers.LSTM(64,return_sequences=0)(lstm1)
# print("lstm2:",lstm2.shape)
# dense1=layers.Dense(64,activation="relu")(lstm2)
# print("dense:",dense1.shape)
# dropout=layers.Dropout(rate=0.2)(dense1)
# print("dropout:",dropout.shape)
# ouput_shape=layers.Dense(1,activation="relu")(dropout)
# model=tf.keras.Model(input_shape,ouput_shape)
# model.compile(loss="mean_squared_error",optimizer="Adam",metrics=["mse"])#mse作为l损失函数，采用Adam作为寻优方式
# history=model.fit(x_train,y_train,batch_size=32,epochs=10,validation_split=0.1,verbose=1)
# model.summary()

In [ ]:
import torch
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

print(is_cuda)
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")